In [1]:
%cd ../src
%load_ext autoreload
%autoreload 2

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [95]:
## Hyperparameters
# steps_to_run = [16, 8, 4, 2]
# steps_to_run = [500, 250, 125, 63, 32, 16, 8, 4, 2]
steps_to_run = [1000]
on_all = True
distilled = False
scheduler = 'ddpm'

categories = ['car']

In [ ]:
from torch.utils.data import DataLoader
from dataloaders.shapenet.shapenet_loader import ShapeNet

path = "../data/ShapeNet"

# train_dataset = ShapeNet(path, "train", 2048, categories, load_renders=False)
# test_dataset = ShapeNet(path, "test", 2048, categories, load_renders=True, total=2000, mean=train_dataset.mean, std=train_dataset.std)
# test_dataset = ShapeNet(path, "test", 2048, categories, load_renders=True, total=2000)
# test_dataset = ShapeNet(path, "val", 2048, categories, load_renders=True, total=2000, mean=train_dataset.mean, std=train_dataset.std)
test_dataset = ShapeNet(path, "val", 2048, categories, load_renders=True, total=2000)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=0)

Loading (test) renders for car (02958343):   0%|          | 0/704 [00:00<?, ?it/s]

In [97]:
from utils.hyperparams import load_hyperparams

hparams_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/hparams.yaml'

hparams = load_hyperparams(hparams_path)

In [98]:
model_args = {
    'voxel_size' : hparams['voxel_size'],
    'nfs' : hparams['nfs'], 
    'attn_chans' : hparams['attn_chans'], 
    'attn_start' : hparams['attn_start'], 
    'cross_attn_chans' : hparams['cross_attn_chans'], 
    'cross_attn_start' : hparams['cross_attn_start'], 
    'cross_attn_cond_dim' : hparams['cross_attn_cond_dim'],
}

model = SPVUnet(**model_args)
model = GSPVD(model=model)

In [99]:
model = model.cuda().eval()

In [100]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.helper_functions import process_ckpt

def get_sched(steps, dist, scheduler):
    if scheduler == 'ddim':
        sched = DDIMSparseScheduler(
            beta_min=hparams['beta_min'], 
            beta_max=hparams['beta_max'], 
            steps=steps, 
            init_steps=hparams['n_steps'],
            mode=hparams['mode'],
        )
    elif dist:
        sched = DDPMSparseScheduler(
            beta_min=hparams['beta_min'], 
            beta_max=hparams['beta_max'], 
            steps=steps, 
            init_steps=hparams['n_steps'],
            mode=hparams['mode'],
        )
    else:
        sched = DDPMSparseScheduler(
            beta_min=hparams['beta_min'], 
            beta_max=hparams['beta_max'], 
            steps=steps, 
            init_steps=steps,
            mode=hparams['mode'],
        )
    return sched

def get_ckpt(steps, dist, scheduler):
    if dist:
        ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/new/{steps}-steps.ckpt'
    elif scheduler == 'ddim':
        ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/1000-steps.ckpt'
    else:
        ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/1000-steps.ckpt'

    ckpt = torch.load(ckpt_path, weights_only=False)
    ckpt = process_ckpt(ckpt)
    return ckpt

In [101]:
from tqdm.auto import tqdm
from metrics.chamfer_dist import ChamferDistanceL2
from metrics.PyTorchEMD.emd import earth_mover_distance as EMD
from utils.helper_functions import normalize_to_unit_sphere, standardize, normalize_to_unit_cube

def run_test(steps):
    CD = ChamferDistanceL2()
    
    sched = get_sched(steps, distilled, scheduler)

    ckpt = get_ckpt(steps, distilled, scheduler)
    model.load_state_dict(ckpt)
    model.eval()

    cd_mean = 0
    emd_mean = 0
    n = 0
    for datapoint in tqdm(test_loader):
        ref_pc = datapoint['pc'].cuda()
        features = datapoint['render-features'].cuda()

        B, N, C = ref_pc.shape
        gen_pc = sched.sample(model, B, N, reference=features, guidance_scale=1)
        
        ref_pc_zero = ref_pc - ref_pc.mean(dim=1, keepdim=True)
        gen_pc_zero = gen_pc - gen_pc.mean(dim=1, keepdim=True)
        
        cd_mean += CD(ref_pc_zero, gen_pc_zero).item() * B
        # emd_mean += EMD(ref_pc, gen_pc, transpose=False).sum().item()
        
        n += B
        print(cd_mean/n, CD(ref_pc_zero, gen_pc_zero).item())
        # print(emd_mean/n)
        
    cd_mean /= n
    emd_mean /= n
    
    return (cd_mean, emd_mean)
    

In [ ]:
import os

def save_means(means, steps):
    path = f'../metrics/{"-".join(categories)}/{scheduler}/{"distilled" if distilled else "skip"}/means/'
    os.makedirs(os.path.dirname(path), exist_ok=True)

    filename = f"{path}/means_{steps}.res"
    string = ""
    for i, ((cd, emd), (cd_norm, emd_norm), (cd_std, emd_std)) in enumerate(means):
        string += f"Steps: {steps:4d}\n"
        string += f"CD: {cd:.8f} | CD (norm): {cd_norm:.8f} | CD (std): {cd_std:.8f}\n"
        string += f"EMD: {emd:.8f} | EMD (norm): {emd_norm:.8f} | EMD (std): {emd_std:.8f}\n"
        string += "-" * 50 + "\n"
    best_cd = min(means, key=lambda x: x[0][0])[0][0]
    best_emd = min(means, key=lambda x: x[0][1])[0][1]
    best_cd_norm = min(means, key=lambda x: x[1][0])[1][0]
    best_emd_norm = min(means, key=lambda x: x[1][1])[1][1]
    best_cd_std = min(means, key=lambda x: x[2][0])[2][0]
    best_emd_std = min(means, key=lambda x: x[2][1])[2][1]
    
    string += f"Best CD: {best_cd:.8f} | Best CD (norm): {best_cd_norm:.8f} | Best CD (std): {best_cd_std:.8f}\n"
    string += f"Best EMD: {best_emd:.8f} | Best EMD (norm): {best_emd_norm:.8f} | Best EMD (std): {best_emd_std:.8f}\n"
    
    with open(filename, "w") as f:
        f.write(string)
        
    print(f"Saved means to {filename}")

for steps in steps_to_run:
    means = [run_test(steps) for _ in range(1)]
    print(means)
    # save_means(means, steps)

  0%|          | 0/11 [00:00<?, ?it/s]

Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

0.05175861716270447 0.05175861716270447


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

0.05134309455752373 0.05092757195234299


...

Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

0.05929878566946302 0.055933836847543716


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

0.06568305753171444 0.11037296056747437


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]